In [1]:
import os


In [3]:
os.chdir('/content/drive/MyDrive/12ab')

In [ ]:
ls

 1.jpeg             c.jpg                    fff.jpg
 a111.jpg           class.jpg                pics/
 a112.jpg           cls.jpg                  ppp.jpg
 a2.jpg             cool.jpg                 recognized_names.txt
 aaaa.jpg           cs.jpg                   sab.jpg
 abc.jpg            d.jpg                    sas.jpg
 a.jpg              e.jpg                    sss.jpg
 arsal_haseeb.jpg   Face_Recognition.ipynb   Testpic
 b1.jpg             faces_detected1.jpg      test.py
 b2.jpg             faces_detected.jpg      'WhatsApp Image 2024-04-28 at 02.20.42_d374a1c4.jpg'
 bbb.jpg            faces.jpg               'WhatsApp Image 2024-05-01 at 12.07.54_d9e707f5.jpg'
 b.jpg              faces_recognized.jpg    'WhatsApp Image 2024-05-01 at 12.15.57_6922bf93.jpg'


In [20]:
import os
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

# Initialize face detector and feature extractor
detector = MTCNN()
embedder = FaceNet()

# Define the similarity threshold for face recognition
THRESHOLD = 1.0 # Adjust this value as needed

# Path to the folder containing subfolders with known faces
known_faces_dir = 'pics'
known_faces = {}

In [5]:
!pip install keras-facenet

  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10369 sha256=ad866b6db440a3a9379ee8e1c985593c96f8df4580e0bd2406eeabb977199a4c
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [2]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.2 MB/s eta 0:00:00


In [7]:
# Load known faces and compute their embeddings
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    if not os.path.isdir(person_dir):
        continue

    for filename in os.listdir(person_dir):
        filepath = os.path.join(person_dir, filename)
        image = cv2.imread(filepath)
        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(image_rgb)

        for face in faces:
            x, y, width, height = face['box']
            face_crop = image_rgb[y:y+height, x:x+width]

            # Check if the crop contains enough pixels
            if face_crop.size == 0:
                continue

            face_features = embedder.embeddings([face_crop])[0]
            known_faces[person_name] = known_faces.get(person_name, [])
            known_faces[person_name].append(face_features)

# Compute the mean of feature vectors for each person
mean_known_faces = {name: np.mean(features, axis=0) for name, features in known_faces.items()}

1/1 [==============================] - 0s 97ms/step


In [21]:
# Load the image in which to recognize faces
image_path = '/content/drive/MyDrive/12ab/Testpic'
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Unable to open image {image_path}")
else:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(image_rgb)
    recognized_names = set()

    for face in faces:
        x, y, width, height = face['box']
        face_crop = image_rgb[y:y+height, x:x+width]

        if face_crop.size == 0:
            continue

        face_features = embedder.embeddings([face_crop])[0]

        min_distance = float('inf')
        recognized_name = 'Unknown'

        for name, known_features in mean_known_faces.items():
            distance = np.linalg.norm(face_features - known_features)
            print(f"Distance to {name}: {distance}")
            if distance < min_distance:
                min_distance = distance
                recognized_name = name

        if min_distance > THRESHOLD:
            recognized_name = 'Unknown'

        print(f"Recognized face: {recognized_name} with distance {min_distance}")
        cv2.rectangle(image, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(image, recognized_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        if recognized_name != 'Unknown':
            recognized_names.add(recognized_name)
with open('recognized_names.txt', 'w') as f:
        for name in recognized_names:
            f.write(f"{name}\n")

1/1 [==============================] - 4s 4s/step
Distance to tehrim: 1.061937928199768
Distance to Ahmed Ali Abid: 1.2082117795944214
Distance to Itba: 1.238444209098816
Distance to Hamza Wajid: 1.2261967658996582
Distance to Ibrahim Hamza: 1.184984564781189
Distance to Zaid bin Muzammil: 1.0066137313842773
Distance to M. Abdullah: 1.1914968490600586
Distance to Umer tayyab: 1.1812642812728882
Distance to Ahmad Farid : 1.1603971719741821
Distance to Rehan riaz: 1.0025056600570679
Distance to Sheharyar: 0.938787579536438
Distance to Hammad anwer: 1.297745943069458
Distance to Adeen amir: 1.089637279510498
Distance to Basim: 1.216454029083252
Distance to Lailoma: 1.0958456993103027
Distance to Eman: 1.2178343534469604
Distance to Omar Khan cs: 1.0423308610916138
Distance to Sarmad: 1.1690584421157837
Distance to Haya: 1.1670781373977661
Distance to Arsal: 1.1443620920181274
Distance to Akbar Jan: 1.299612283706665
Distance to AbdurRahman Durrani: 1.3337171077728271
Distance to Raja: 0.9

In [22]:
cv2.imwrite('a1.jpg', image_display)





True

In [18]:
image_display = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)